# Evolver Loop 2 - LB Feedback Analysis

**LB Score: 117.2815** (CV: 117.2815, gap: 0.0000)

The CV-LB gap is essentially zero, which is expected for this optimization problem (no train/test split).

## Key Evaluator Feedback:
1. **ENSEMBLE APPROACH NOT IMPLEMENTED** - This is the #1 technique in top kernels
2. **No external solution datasets used** - Pre-computed solutions may already be better
3. **bbox3 binary not used** - Primary optimizer in winning solutions
4. **Limited optimization time** - Top kernels use 3-hour budgets

## Analysis Goals:
1. Scan all available snapshot CSVs and score them
2. Build an ensemble baseline from best configurations
3. Identify the gap between ensemble baseline and target

In [ ]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
import os
import glob
from tqdm import tqdm

getcontext().prec = 25
scale_factor = Decimal("1e18")

print("Libraries loaded")

In [ ]:
# ChristmasTree class
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

print("ChristmasTree class defined")

In [ ]:
# Fast scoring using numpy (no Shapely for speed)
def fast_score_group(xs, ys, degs, n):
    """Fast scoring using numpy - no polygon construction needed for bounding box."""
    # Tree template vertices
    tx = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
    ty = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])
    
    all_x = []
    all_y = []
    
    for i in range(len(xs)):
        rad = np.radians(degs[i])
        c, s = np.cos(rad), np.sin(rad)
        # Rotate and translate
        rx = c * tx - s * ty + xs[i]
        ry = s * tx + c * ty + ys[i]
        all_x.extend(rx)
        all_y.extend(ry)
    
    all_x = np.array(all_x)
    all_y = np.array(all_y)
    
    side = max(all_x.max() - all_x.min(), all_y.max() - all_y.min())
    return side**2 / n

def strip_s(val):
    """Remove 's' prefix from string values."""
    s = str(val)
    return float(s[1:] if s.startswith('s') else s)

def score_csv(filepath):
    """Score a submission CSV file."""
    try:
        df = pd.read_csv(filepath)
        if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
            return None, None
        
        df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
        
        total_score = 0.0
        scores_per_n = {}
        
        for n, group in df.groupby('N'):
            if n < 1 or n > 200:
                continue
            xs = np.array([strip_s(v) for v in group['x']])
            ys = np.array([strip_s(v) for v in group['y']])
            degs = np.array([strip_s(v) for v in group['deg']])
            
            if len(xs) != n:
                continue
                
            score = fast_score_group(xs, ys, degs, n)
            scores_per_n[n] = score
            total_score += score
        
        return total_score, scores_per_n
    except Exception as e:
        return None, None

print("Scoring functions defined")

In [ ]:
# Find all available CSV files
csv_files = []

# Our current candidates
csv_files.extend(glob.glob('/home/code/submission_candidates/*.csv'))

# Snapshot files
csv_files.extend(glob.glob('/home/nonroot/snapshots/santa-2025/*/code/submission_candidates/*.csv'))
csv_files.extend(glob.glob('/home/nonroot/snapshots/santa-2025/*/code/*.csv'))
csv_files.extend(glob.glob('/home/nonroot/snapshots/santa-2025/*/submission/*.csv'))
csv_files.extend(glob.glob('/home/nonroot/snapshots/santa-2025/*/code/experiments/*/*.csv'))

# Remove duplicates
csv_files = list(set(csv_files))
print(f"Found {len(csv_files)} CSV files to analyze")

In [ ]:
# Score all CSV files
results = []

for filepath in tqdm(csv_files, desc="Scoring CSVs"):
    total_score, scores_per_n = score_csv(filepath)
    if total_score is not None:
        results.append({
            'filepath': filepath,
            'total_score': total_score,
            'scores_per_n': scores_per_n
        })

# Sort by score
results.sort(key=lambda x: x['total_score'])

print(f"\nSuccessfully scored {len(results)} files")
print("\nTop 10 best scores:")
for i, r in enumerate(results[:10]):
    print(f"{i+1}. {r['total_score']:.6f} - {r['filepath'].split('/')[-1]}")

In [ ]:
# Build ensemble: for each N, pick the best configuration across all sources
print("\n" + "="*60)
print("BUILDING ENSEMBLE BASELINE")
print("="*60)

best_per_n = {n: {'score': float('inf'), 'source': None, 'data': None} for n in range(1, 201)}

for r in results:
    for n, score in r['scores_per_n'].items():
        if score < best_per_n[n]['score']:
            best_per_n[n]['score'] = score
            best_per_n[n]['source'] = r['filepath']

# Calculate ensemble score
ensemble_score = sum(best_per_n[n]['score'] for n in range(1, 201))
print(f"\nEnsemble baseline score: {ensemble_score:.6f}")
print(f"Target score: 68.931058")
print(f"Gap to target: {ensemble_score - 68.931058:.6f}")

# Show source distribution
from collections import Counter
sources = Counter([best_per_n[n]['source'].split('/')[-1] for n in range(1, 201)])
print(f"\nSources used in ensemble:")
for src, count in sources.most_common(10):
    print(f"  {src}: {count} configurations")

In [ ]:
# Compare our best vs ensemble
print("\n" + "="*60)
print("COMPARISON")
print("="*60)

our_best = 117.281454
print(f"Our best score: {our_best:.6f}")
print(f"Ensemble score: {ensemble_score:.6f}")
print(f"Improvement from ensemble: {our_best - ensemble_score:.6f}")
print(f"\nTarget: 68.931058")
print(f"Gap from ensemble to target: {ensemble_score - 68.931058:.6f} ({(ensemble_score - 68.931058)/68.931058*100:.1f}%)")

In [ ]:
# Analyze which N values have the most room for improvement
print("\n" + "="*60)
print("N VALUES WITH LARGEST GAPS")
print("="*60)

# Theoretical minimum for N=1 is a single tree at 45 degrees
# side = sqrt(2) * 0.8 (approximately)
theoretical_n1 = (np.sqrt(2) * 0.8)**2 / 1  # ~1.28

print(f"\nTheoretical minimum for N=1: ~{theoretical_n1:.4f}")
print(f"Current best for N=1: {best_per_n[1]['score']:.6f}")

# Show worst N values
worst_n = sorted([(n, best_per_n[n]['score']) for n in range(1, 201)], key=lambda x: -x[1])[:20]
print(f"\nTop 20 worst N values (highest score contribution):")
for n, score in worst_n:
    print(f"  N={n}: {score:.6f}")